In [1]:
%%capture
!pip install transformers
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%%bash
cd input-marginalization
git pull
cd ..

Updating 9d08737..6630b06
Fast-forward
 metrics.py | 6 ++----
 1 file changed, 2 insertions(+), 4 deletions(-)


From https://github.com/ronakdm/input-marginalization
   9d08737..6630b06  main       -> origin/main


In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
save_dir = "/content/gdrive/My Drive/input-marginalization"

Mounted at /content/gdrive


In [9]:
%%capture
import sys
sys.path.append("input-marginalization")

from metrics import input_marginalization, color_sentence
import torch
from transformers import BertForMaskedLM

In [5]:
%%capture
mlm = BertForMaskedLM.from_pretrained('bert-base-uncased').cuda()

In [6]:
bert = torch.load(f"{save_dir}/bert_sst2.pt")
cnn = torch.load(f"{save_dir}/cnn_sst2.pt")
lstm = torch.load(f"{save_dir}/lstm_sst2.pt")

In [7]:
lstm_sentences = [
  ("pacino is brilliant as the sleep - deprived dormer , his increasing weariness as much existential as it is physical .", 1),
  ("an important movie , a reminder of the power of file to move us and to make us examine our values .", 1),
  ("unflinchingly bleak and desperate", 0),
  ("i am sorry that i was unable to get the full brunt of this comedy .", 0),
]
bert_sentences = [
  ("it ' s a lovely film with lovely performances by buy and accorsi .", 1),
  ("more romantic , emotional and ultimately more satisfying than the teary - eyed original .", 1),
  ("it ' s a bit disappointing that it only manages to be decent instead of dead brilliant .", 0),
  ("suffers from the lack of a compelling or comprehensible narrative .", 0),   
]
cnn_sentences = [
  (". . . very funny , very enjoyable . . .", 1),
  ("lookin ' for sin , american - style ?", 0),
]

In [10]:
for sentence, label in lstm_sentences:
  att_scores = input_marginalization(lstm, sentence, mlm, target_label=label)
  color_sentence(sentence, att_scores)

In [ ]:
try:
  for sentence, label in bert_sentences:
    att_scores = input_marginalization(bert, sentence, mlm, target_label=label)
    color_sentence(sentence, att_scores)
except KeyboardInterrupt:
  print("Graceful Exit")

In [11]:
for sentence, label in cnn_sentences:
  att_scores = input_marginalization(cnn, sentence, mlm, target_label=label)
  color_sentence(sentence, att_scores)